In [1]:
import pandas as pd
import numpy as np

dates=pd.date_range('2017',periods = 120, freq = 'Q')
sigma = 0.1
mu = 1
dataseries=pd.Series(sigma * np.random.randn(120) + mu,index=dates)
df=pd.DataFrame({'Test':dataseries})
df.head()

,Test
2017-03-31,0.938993
2017-06-30,1.035451
2017-09-30,0.843064
2017-12-31,0.990759
2018-03-31,0.878855


In [2]:
s1 = df['Test']

In [3]:
def calculate_ratio(ts1, ts2):
    '''
    '''
    return ts1 / ts2


def calculate_difference(ts):
    '''
    '''
    return ts - ts.shift(1)

def calculate_transform(ts):
    '''
    '''
    return ts / ts.shift(1) - 1

def calculate_inverse_difference(ts, start): 
    '''
    '''
    vals = [start]*len(ts)  # init fixed size array
    for i in range(1, len(ts)):
        vals[i] = vals[i-1] + ts.iloc[i]
    return pd.Series(vals, ts.index)

def calculate_inverse_transform(ts, start): 
    '''
    '''
    vals = [start]*len(ts)  # init fixed size array
    for i in range(1, len(ts)):
        vals[i] = vals[i-1] * (1 + ts.iloc[i])
    return pd.Series(vals, ts.index)

In [4]:
s2 = calculate_difference(s1)
start = s1.iloc[0]
s3 = calculate_inverse_difference(s2, start)
s4 = calculate_transform(s1)
s5 = calculate_inverse_transform(s4, start)

In [5]:
s1

2017-03-31    0.938993
2017-06-30    1.035451
2017-09-30    0.843064
2017-12-31    0.990759
2018-03-31    0.878855
2018-06-30    1.014929
2018-09-30    1.111341
2018-12-31    1.148558
2019-03-31    0.883348
2019-06-30    0.963354
2019-09-30    0.939477
2019-12-31    0.897250
2020-03-31    1.037336
2020-06-30    0.960818
2020-09-30    0.890565
2020-12-31    0.923647
2021-03-31    0.816016
2021-06-30    1.090655
2021-09-30    0.966782
2021-12-31    0.906063
2022-03-31    0.973085
2022-06-30    0.910170
2022-09-30    1.067559
2022-12-31    1.108677
2023-03-31    0.978722
2023-06-30    0.990711
2023-09-30    0.911618
2023-12-31    0.974311
2024-03-31    1.015359
2024-06-30    1.002178
                ...   
2039-09-30    1.097034
2039-12-31    1.087613
2040-03-31    0.995884
2040-06-30    1.128371
2040-09-30    0.835371
2040-12-31    0.843538
2041-03-31    1.030430
2041-06-30    1.105285
2041-09-30    0.958451
2041-12-31    1.085689
2042-03-31    0.974051
2042-06-30    1.099131
2042-09-30 

In [6]:
s2

2017-03-31         NaN
2017-06-30    0.096457
2017-09-30   -0.192387
2017-12-31    0.147695
2018-03-31   -0.111905
2018-06-30    0.136075
2018-09-30    0.096411
2018-12-31    0.037217
2019-03-31   -0.265209
2019-06-30    0.080006
2019-09-30   -0.023877
2019-12-31   -0.042227
2020-03-31    0.140086
2020-06-30   -0.076517
2020-09-30   -0.070254
2020-12-31    0.033082
2021-03-31   -0.107631
2021-06-30    0.274639
2021-09-30   -0.123873
2021-12-31   -0.060719
2022-03-31    0.067023
2022-06-30   -0.062915
2022-09-30    0.157389
2022-12-31    0.041118
2023-03-31   -0.129955
2023-06-30    0.011989
2023-09-30   -0.079093
2023-12-31    0.062693
2024-03-31    0.041049
2024-06-30   -0.013182
                ...   
2039-09-30   -0.002440
2039-12-31   -0.009422
2040-03-31   -0.091729
2040-06-30    0.132487
2040-09-30   -0.293000
2040-12-31    0.008167
2041-03-31    0.186891
2041-06-30    0.074855
2041-09-30   -0.146834
2041-12-31    0.127237
2042-03-31   -0.111637
2042-06-30    0.125079
2042-09-30 

In [7]:
s3

2017-03-31    0.938993
2017-06-30    1.035451
2017-09-30    0.843064
2017-12-31    0.990759
2018-03-31    0.878855
2018-06-30    1.014929
2018-09-30    1.111341
2018-12-31    1.148558
2019-03-31    0.883348
2019-06-30    0.963354
2019-09-30    0.939477
2019-12-31    0.897250
2020-03-31    1.037336
2020-06-30    0.960818
2020-09-30    0.890565
2020-12-31    0.923647
2021-03-31    0.816016
2021-06-30    1.090655
2021-09-30    0.966782
2021-12-31    0.906063
2022-03-31    0.973085
2022-06-30    0.910170
2022-09-30    1.067559
2022-12-31    1.108677
2023-03-31    0.978722
2023-06-30    0.990711
2023-09-30    0.911618
2023-12-31    0.974311
2024-03-31    1.015359
2024-06-30    1.002178
                ...   
2039-09-30    1.097034
2039-12-31    1.087613
2040-03-31    0.995884
2040-06-30    1.128371
2040-09-30    0.835371
2040-12-31    0.843538
2041-03-31    1.030430
2041-06-30    1.105285
2041-09-30    0.958451
2041-12-31    1.085689
2042-03-31    0.974051
2042-06-30    1.099131
2042-09-30 

In [8]:
s4

2017-03-31         NaN
2017-06-30    0.102724
2017-09-30   -0.185800
2017-12-31    0.175189
2018-03-31   -0.112948
2018-06-30    0.154832
2018-09-30    0.094993
2018-12-31    0.033488
2019-03-31   -0.230906
2019-06-30    0.090571
2019-09-30   -0.024785
2019-12-31   -0.044948
2020-03-31    0.156128
2020-06-30   -0.073763
2020-09-30   -0.073119
2020-12-31    0.037147
2021-03-31   -0.116528
2021-06-30    0.336561
2021-09-30   -0.113577
2021-12-31   -0.062806
2022-03-31    0.073971
2022-06-30   -0.064656
2022-09-30    0.172923
2022-12-31    0.038516
2023-03-31   -0.117216
2023-06-30    0.012249
2023-09-30   -0.079835
2023-12-31    0.068771
2024-03-31    0.042131
2024-06-30   -0.012982
                ...   
2039-09-30   -0.002220
2039-12-31   -0.008588
2040-03-31   -0.084340
2040-06-30    0.133035
2040-09-30   -0.259666
2040-12-31    0.009777
2041-03-31    0.221556
2041-06-30    0.072645
2041-09-30   -0.132847
2041-12-31    0.132753
2042-03-31   -0.102826
2042-06-30    0.128411
2042-09-30 

In [9]:
s5

2017-03-31    0.938993
2017-06-30    1.035451
2017-09-30    0.843064
2017-12-31    0.990759
2018-03-31    0.878855
2018-06-30    1.014929
2018-09-30    1.111341
2018-12-31    1.148558
2019-03-31    0.883348
2019-06-30    0.963354
2019-09-30    0.939477
2019-12-31    0.897250
2020-03-31    1.037336
2020-06-30    0.960818
2020-09-30    0.890565
2020-12-31    0.923647
2021-03-31    0.816016
2021-06-30    1.090655
2021-09-30    0.966782
2021-12-31    0.906063
2022-03-31    0.973085
2022-06-30    0.910170
2022-09-30    1.067559
2022-12-31    1.108677
2023-03-31    0.978722
2023-06-30    0.990711
2023-09-30    0.911618
2023-12-31    0.974311
2024-03-31    1.015359
2024-06-30    1.002178
                ...   
2039-09-30    1.097034
2039-12-31    1.087613
2040-03-31    0.995884
2040-06-30    1.128371
2040-09-30    0.835371
2040-12-31    0.843538
2041-03-31    1.030430
2041-06-30    1.105285
2041-09-30    0.958451
2041-12-31    1.085689
2042-03-31    0.974051
2042-06-30    1.099131
2042-09-30 